<a href="https://colab.research.google.com/github/SeanBarnier/HAFS_Air-Sea/blob/main/miltonAnimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Set up environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install cartopy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cft
import pandas as pd
from datetime import datetime as dt

Set user parameters

In [ ]:
name = "Milton"
tcNum = "14"
trackType = ""

initTime = dt(year=2024, month=10, day=7, hour=18) #Time when Milton began its most rapid intensification
initStr, initHour = initTime.strftime("%Y%m%d_%H").split("_")

fHourStep = 3      #Normally 3 for HAFS-A
forecastLength = 30 #Normally 126 for HAFS-A.
#runStep = 6         #Normally 6 for HAFS-A

figureSuffix = "_RI"
subfolder = "RI/"
dataPath = "/content/drive/MyDrive/savedData/"
figurePath = "/content/drive/MyDrive/figures/"

potentialTemp = True #Use atmospheric potential temperature instead of in-situ temperature

lonMin, lonMax, latMin, latMax = -92, -85, 21, 25
atmLayer = 1000.0

Find times needed

In [ ]:
dateFormat = "%Y-%m-%d %H:%M:%S"
runFormat = "%Y%m%d%H"

#Times to plot
fcastTimes = [] #Key: initiation, item: valid time list
fhour = 0
validTime = initTime
while fhour <= forecastLength:
    fcastTimes.append(validTime)
    validTime += pd.Timedelta(hours=fHourStep)
    fhour += fHourStep

Create animation

In [ ]:
atmFig, atmAx = plt.subplots(figsize=(10, 10), subplot_kw={"projection": ccrs.PlateCarree()})
contourLevs = np.linspace(0, 80, 17)

def update(valid):

  fhour = str(int((valid-initTime).total_seconds() / 3600))
  while len(fhour) < 3: fhour = "0" + fhour

  atmFile = "hafsa_" + initStr + initHour + "_f" + fhour + ".nc"
  atmPath = dataPath + "hafsaOutput/" + subfolder + atmFile
  atmData = xr.open_dataset(atmPath)

  atmSlice = atmData.sel(isobaricInhPa=atmLayer).sel(longitude=slice(lonMin+360, lonMax+360), latitude=slice(latMin, latMax))
  dat = np.sqrt(atmSlice.u.data ** 2 + atmSlice.v.data ** 2)

  tempContour = atmAx.contourf(atmSlice.longitude.data, atmSlice.latitude.data, dat, cmap="viridis", transform=ccrs.PlateCarree(),
                              extent = [lonMin, lonMax, latMin, latMax], levels=contourLevs)
  atmAx.coastlines()
  atmAx.gridlines(draw_labels=["left", "bottom"], alpha=0.5)
  atmAx.set_title(valid.strftime("%Y-%m-%d %HUTC"))

  atmAx.set_extent([lonMin, lonMax, latMin, latMax])

  return [tempContour] # Return a list of artists

def init():
  tempContour = update(fcastTimes[0])
  atmFig.colorbar(tempContour[0], shrink=0.4, label="Wind Speed (ms$^{-1}$)")
  return tempContour

ani = FuncAnimation(atmFig, update, frames=fcastTimes,
                    init_func=init, blit=True)
ani.save("miltonWind.gif", fps=1.5)
plt.show()